In [1]:
import pandas as pd
import math
import events
import os
import dataset
import numpy as np
import timestamps

MINUTES = 60 * 4
SRC_DIR = 'dataset/bob_all_processed_mins'
MINUTES_DIR = f'dataset/transformed_minutes/interval_{str(MINUTES)}m'
TIMESTAMP_DIR = f'dataset/transformed_timestamps/interval_{str(MINUTES)}m'
POPULATED_DIR = f'dataset/transformed_populated/interval_{str(MINUTES)}m'
FILLED_DIR = f'dataset/transformed_filled/interval_{str(MINUTES)}m'
TIMESTAMPS = timestamps.get_all_timestamps()

os.makedirs(MINUTES_DIR, exist_ok=True)
os.makedirs(POPULATED_DIR, exist_ok=True)
os.makedirs(FILLED_DIR, exist_ok=True)
os.makedirs(TIMESTAMP_DIR, exist_ok=True)

files = os.scandir(SRC_DIR)

# Transform to N minutes

In [2]:
for file in files:
    df = pd.read_csv(file.path, dtype={
        't_i_1': float,
        't_i_2': float,
        't_i_3': float,
        't_i_4': float,
        't_o': float,
        'weight_kg': float,
        "weight_delta": float,
        'numeric.time': float,
        'h': float,
        't': float,
        'p': float,
    }, low_memory=False, parse_dates=['time'], index_col='time', date_format='%Y-%m-%d %H:%M:%S')

    df: pd.DataFrame = df.infer_objects(copy=False)

    
    print(f'Processing {file.name}: ', end='')
    
    has_queencell = events.populate_event_column(df, 'queencell')
    has_feeding = events.populate_event_column(df, 'feeding')
    has_honey = events.populate_event_column(df, 'honey')
    has_treatment = events.populate_event_column(df, 'treatment')
    has_died = events.populate_event_column(df, 'died')
    has_swarming = events.populate_event_column(df, 'swarming')
    
    # if has_queencell:
    #     print(f' Queencell')
    if has_feeding:
        print(f'Feeding', end=' ')
    if has_honey:
        print(f'Honey', end=' ')
    if has_treatment:
        print(f'Treatment', end=' ')
    if has_died:
        print(f'Died', end=' ')
    if has_swarming:
        print(f'Swarming', end=' ')
    print()
    
    # odf = pd.DataFrame(columns=columns)
    odf = df.resample(f'{MINUTES}min').agg({
        'X.1': 'first',
        'X': 'first',
        'key': 'first',

        't_i_1': 'mean',
        't_i_2': 'mean',
        't_i_3': 'mean',
        't_i_4': 'mean',
        't_i_5': 'mean',
        't_o': 'mean',
        
        'weight_kg': 'mean',
        
        'h': 'mean',
        't': 'mean',
        'p': 'mean',
        
        'year': 'first',
        'month': 'first',
        'day': 'first',
        'hour': 'first',
        'minute': 'first',
        
        'queencell.next.dif': 'first',
        'feeding.next.dif': 'first',
        'honey.next.dif': 'first',
        'treatment.next.dif': 'first',
        'died.next.dif': 'first',
        'swarming.next.dif': 'first',
        
        'swarming': 'max',
        'queencell': 'max',
        'feeding': 'max',
        'honey': 'max',
        'treatment': 'max',
        'died': 'max',
    })
    
    # Convert columns to integer type before saving
    odf['month'] = odf['month'].fillna(0).astype(int)
    odf['day'] = odf['day'].fillna(0).astype(int)
    odf['year'] = odf['year'].fillna(0).astype(int)
    odf['hour'] = odf['hour'].fillna(0).astype(int)
    odf['minute'] = odf['minute'].fillna(0).astype(int)
    odf['swarming'] = odf['swarming'].fillna(0).astype(int)
    odf['feeding'] = odf['feeding'].fillna(0).astype(int)
    odf['honey'] = odf['honey'].fillna(0).astype(int)
    odf['treatment'] = odf['treatment'].fillna(0).astype(int)
    odf['died'] = odf['died'].fillna(0).astype(int)
    odf['queencell'] = odf['queencell'].fillna(0).astype(int)
    
    odf = odf.round(2)
    odf.drop(columns=['X.1', 'X', 'key', 'queencell.next.dif','feeding.next.dif','honey.next.dif','treatment.next.dif','died.next.dif','swarming.next.dif'], inplace=True)
    # odf['time'] = odf.index
    odf.to_csv(f'{MINUTES_DIR}/{file.name}', index=True, index_label='time')



Processing 2020_79.csv: Feeding Treatment Swarming 
Processing 2022_97.csv: 
Processing 2022_152.csv: Honey 
Processing 2022_82.csv: 
Processing 2022_96.csv: 
Processing 2022_69.csv: 
Processing 2020_87.csv: Honey 
Processing 2020_93.csv: 
Processing 2020_123.csv: Feeding 
Processing 2020_78.csv: 
Processing 2021_132.csv: 
Processing 2021_126.csv: Feeding Honey Treatment 
Processing 2021_118.csv: Swarming 
Processing 2020_85.csv: Honey 
Processing 2020_109.csv: Feeding Honey Treatment 
Processing 2022_57.csv: 
Processing 2022_151.csv: 
Processing 2022_95.csv: 
Processing 2022_56.csv: 
Processing 2020_90.csv: Feeding Honey Treatment 
Processing 2020_84.csv: 
Processing 2020_120.csv: 
Processing 2020_47.csv: 
Processing 2021_109.csv: 
Processing 2021_135.csv: Feeding 
Processing 2020_43.csv: 
Processing 2020_57.csv: 
Processing 2020_118.csv: Feeding Treatment Swarming 
Processing 2020_80.csv: 
Processing 2022_141.csv: 
Processing 2020_95.csv: 
Processing 2020_119.csv: Feeding Treatment S

# Export to timestamps

In [3]:
for stamp in TIMESTAMPS:
    name = f"{stamp['year']}_{stamp['hive_number']}.csv"
    print(f"Processing {name}")
    df = dataset.read_dataset_file(os.path.join(MINUTES_DIR, name))
    if 'ignore' in stamp:
        df[[stamp['ignore']]] = None
    date_from: pd.Timestamp = pd.to_datetime(stamp['date_from'])
    date_to:pd.Timestamp = pd.to_datetime(stamp['date_to'])
    
    # if stamp['year'] == '2022' and stamp['hive_number'] == '88':
    #     mask = df.index < pd.Timestamp('2022-02-22 18:00')
    #     df.index = df.index + pd.Timedelta(days=9, hours=1).where(mask, pd.Timedelta(0))
    #     date_from = df.index.min()
        
    df = df.loc[date_from:date_to]
    out_name = f"{stamp['year']}_{stamp['hive_number']}__{date_from.month}-{date_from.day}={date_to.month}-{date_to.day}.csv"
    df.to_csv(os.path.join(TIMESTAMP_DIR, out_name), index=True)

Processing 2019_5.csv
Processing 2019_5.csv
Processing 2020_48.csv
Processing 2020_48.csv
Processing 2020_105.csv
Processing 2019_49.csv
Processing 2019_56.csv
Processing 2019_58.csv
Processing 2020_0.csv
Processing 2020_26.csv
Processing 2020_27.csv
Processing 2020_36.csv
Processing 2020_47.csv
Processing 2020_48.csv
Processing 2020_66.csv
Processing 2020_69.csv
Processing 2020_72.csv
Processing 2020_72.csv
Processing 2020_76.csv
Processing 2020_84.csv
Processing 2020_89.csv
Processing 2020_90.csv
Processing 2020_95.csv
Processing 2020_96.csv
Processing 2020_96.csv
Processing 2020_97.csv
Processing 2020_97.csv
Processing 2020_100.csv
Processing 2021_0.csv
Processing 2020_105.csv
Processing 2020_105.csv
Processing 2020_107.csv
Processing 2020_111.csv
Processing 2020_111.csv
Processing 2020_111.csv
Processing 2020_112.csv
Processing 2020_118.csv
Processing 2020_119.csv
Processing 2021_21.csv
Processing 2020_123.csv
Processing 2020_123.csv
Processing 2021_27.csv
Processing 2020_128.csv
P

# Fill missing values

In [4]:
files = os.scandir(TIMESTAMP_DIR)
for file in files:
    df = dataset.read_dataset_file(file.path)
    if '2020_118__5-28=6-9' in file.name:
        df = dataset.fill_with_historical_pattern(df, ['t_i_1', 't_i_2', 't_i_3', 't_i_4', 't_i_5', 't_o', 't', 'weight_kg'], hours_ago=48)
    if '2020_112__6-8=6-22' in file.name:
        df = dataset.fill_with_historical_pattern(df, ['t_o'], hours_ago=48)
    if '2022_88__1-' in file.name:
        # Shift records before 2022-02-22 18:00 forward by 9 days and 1 hour
        cutoff_date = pd.to_datetime('2022-02-22 18:00')
        time_shift = pd.Timedelta(days=9, hours=1)
        # Create mask for records before cutoff
        mask = df.index < cutoff_date
        # Apply time shift to matching records
        df.index = pd.to_datetime(np.where(mask, 
                                        df.index + time_shift,
                                        df.index))
        # Sort index after shifting
        df.sort_index(inplace=True)
    df.infer_objects(copy=False)
    df.bfill()
    df.to_csv(os.path.join(FILLED_DIR, file.name), index=True, index_label='time')

# Populate additional columns

In [5]:
files = os.scandir(FILLED_DIR)
for file in files:
    print(f"Processing {file.name}:", end=' ')
    
    df = dataset.read_dataset_file(file.path)
    df.infer_objects(copy=False)                
    df.bfill()
    print(f"Smoothing", end=' ')
    df['weight_kg_smoothed'] = dataset.smooth_col(df['weight_kg'])
    
    df['temp_mid'] = df[['t_i_1', 't_i_2', 't_i_3', 't_i_4', 't_i_5', "t"]].median(axis=1, skipna=True)
    df['temp_mid_smoothed'] = dataset.smooth_col(df['temp_mid'])
    df['temp_diff'] = df['temp_mid'] - df['t_o']
    df['temp_ratio'] = df['temp_mid'] / df['t_o']
    
    df['h'] = dataset.smooth_col(df['h'])
    df['p'] = dataset.smooth_col(df['p'])
    print(f"Populating", end=' ')
    df['weight_kg_1_pct'] = df['weight_kg_smoothed'].pct_change(periods=1)
    df['weight_kg_2_pct'] = df['weight_kg_smoothed'].pct_change(periods=2)
    df['weight_kg_3_pct'] = df['weight_kg_smoothed'].pct_change(periods=3)
    df['weight_kg_5_pct'] = df['weight_kg_smoothed'].pct_change(periods=5)
    df['weight_kg_8_pct'] = df['weight_kg_smoothed'].pct_change(periods=8)
    
    df['temp_mid_3_pct'] = df['temp_mid_smoothed'].pct_change(periods=3)
    df['temp_mid_5_pct'] = df['temp_mid_smoothed'].pct_change(periods=5)
    df['temp_mid_10_pct'] = df['temp_mid_smoothed'].pct_change(periods=10)
    
    df.to_csv(f'{POPULATED_DIR}/{file.name}', index=True, index_label='time')
    print()

Processing 2020_119__5-28=5-30.csv: Smoothing Populating 
Processing 2021_126__8-14=8-28.csv: Smoothing Populating 
Processing 2021_79__2-22=6-15.csv: Smoothing Populating 
Processing 2020_26__8-28=9-17.csv: Smoothing Populating 
Processing 2021_86__5-21=6-4.csv: Smoothing Populating 
Processing 2020_107__7-21=8-1.csv: Smoothing Populating 
Processing 2021_123__3-1=4-15.csv: Smoothing Populating 
Processing 2020_72__7-9=7-14.csv: Smoothing Populating 
Processing 2021_67__4-2=4-21.csv: Smoothing Populating 
Processing 2019_58__11-8=11-10.csv: Smoothing Populating 
Processing 2020_48__1-1=1-14.csv: Smoothing Populating 
Processing 2019_43__11-26=12-8.csv: Smoothing Populating 
Processing 2022_21__2-15=3-26.csv: Smoothing Populating 
Processing 2020_69__7-16=7-23.csv: Smoothing Populating 
Processing 2020_105__5-28=5-31.csv: Smoothing Populating 
Processing 2022_129__3-1=4-15.csv: Smoothing Populating 
Processing 2020_96__9-15=10-1.csv: Smoothing Populating 
Processing 2020_48__2-1=2-10.c